In [1]:
import numpy as np
import scipy.linalg as slin
import scipy.optimize as sopt
from scipy.special import expit as sigmoid
import networkx as nx
import random
import pandas as pd
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""

def notears_linear_l1(X, lambda1, loss_type, max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3):
    """Solve min_W L(W; X) + lambda1 ‖W‖_1 s.t. h(W) = 0 using augmented Lagrangian.

    Args:
        X (np.ndarray): [n, d] sample matrix
        lambda1 (float): l1 penalty parameter
        loss_type (str): l2, logistic, poisson
        max_iter (int): max num of dual ascent steps
        h_tol (float): exit if |h(w_est)| <= htol
        rho_max (float): exit if rho >= rho_max
        w_threshold (float): drop edge if |weight| < threshold

    Returns:
        W_est (np.ndarray): [d, d] estimated DAG
    """
    def _loss(W):
        """Evaluate value and gradient of loss."""
        M = X @ W
        if loss_type == 'l2':
            R = X - M
            loss = 0.5 / X.shape[0] * (R ** 2).sum()
            G_loss = - 1.0 / X.shape[0] * X.T @ R
        elif loss_type == 'logistic':
            loss = 1.0 / X.shape[0] * (np.logaddexp(0, M) - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (sigmoid(M) - X)
        elif loss_type == 'poisson':
            S = np.exp(M)
            loss = 1.0 / X.shape[0] * (S - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (S - X)
        else:
            raise ValueError('unknown loss type')
        return loss, G_loss

    def _h(W):
        """Evaluate value and gradient of acyclicity constraint."""
        #     E = slin.expm(W * W)  # (Zheng et al. 2018)
        #     h = np.trace(E) - d
        M = np.eye(d) + W * W / d  # (Yu et al. 2019)
        E = np.linalg.matrix_power(M, d - 1)
        h = (E.T * M).sum() - d
        G_h = E.T * W * 2
        return h, G_h

    def _adj(w):
        """Convert doubled variables ([2 d^2] array) back to original variables ([d, d] matrix)."""
        return (w[:d * d] - w[d * d:]).reshape([d, d])

    def _func(w):
        """Evaluate value and gradient of augmented Lagrangian for doubled variables ([2 d^2] array)."""
        W = _adj(w)
        loss, G_loss = _loss(W)
        h, G_h = _h(W)
        # us obj as loss
        obj = loss + 0.5 * rho * h * h + alpha * h + lambda1 * w.sum()
        G_smooth = G_loss + (rho * h + alpha) * G_h
        g_obj = np.concatenate((G_smooth + lambda1, - G_smooth + lambda1), axis=None)
        return obj, g_obj

    n, d = X.shape
    w_est, rho, alpha, h = np.zeros(2 * d * d), 1.0, 0.0, np.inf  # double w_est into (w_pos, w_neg)
    bnds = [(0, 0) if i == j else (0, None) for _ in range(2) for i in range(d) for j in range(d)]
    for _ in range(max_iter):
        w_new, h_new = None, None
        while rho < rho_max:
            sol = sopt.minimize(_func, w_est, method='L-BFGS-B', jac=True, bounds=bnds)
            w_new = sol.x
            h_new, _ = _h(_adj(w_new))
            if h_new > 0.25 * h:
                rho *= 10
            else:
                break
        w_est, h = w_new, h_new
        alpha += rho * h
        if h <= h_tol or rho >= rho_max:
            break
    print(w_est)
    W_est = _adj(w_est)
    W_est[np.abs(W_est) < w_threshold] = 0
    return W_est



In [2]:
def random_dag(nodes, edges):
    """Generate a random Directed Acyclic Graph (DAG) with a given number of nodes and edges."""
    G = nx.DiGraph()
    for i in range(nodes):
        G.add_node(i)
    while edges > 0:
        a = random.randint(0,nodes-1)
        b=a
        while b==a:
            b = random.randint(0,nodes-1)
        G.add_edge(a,b)
        if nx.is_directed_acyclic_graph(G):
            edges -= 1
        else:
            # we closed a loop!
            G.remove_edge(a,b)
    return G

# This function generates data according to a DAG provided in list_vertex and list_edges with mean and variance as input
# It will apply a perturbation at each node provided in perturb.
def gen_data(G, mean = 0, var = 1, SIZE = 10000, perturb = []):
    list_edges = G.edges()
    list_vertex = G.nodes()

    order = []
    for ts in nx.algorithms.dag.topological_sort(G):
        order.append(ts)

    g = []
    for v in list_vertex:
        if v in perturb:
            g.append(np.random.normal(mean,var,SIZE))
            print("perturbing ", v, "with mean var = ", mean, var)
        else:
            g.append(np.random.normal(0,1,SIZE))

    for o in order:
        for edge in list_edges:
            if o == edge[1]: # if there is an edge into this node
                g[edge[1]] += g[edge[0]]
    g = np.swapaxes(g,0,1)
    return pd.DataFrame(g, columns = list(map(str, list_vertex)))


In [3]:
nodes = 5
datasize = 1000
G = random_dag(nodes, nodes*nodes)
df = gen_data(G, SIZE = datasize)

In [5]:
X_DAG = df.values
W_est = notears_linear_l1(X_DAG, lambda1=0.1, loss_type='l2')
W_est

[0.00000000e+00 9.29348164e-05 5.86389088e-06 2.01444849e-05
 4.82854217e-05 4.98242295e-01 0.00000000e+00 7.93205218e-01
 9.13842392e-01 8.87965036e-01 1.24654892e+00 3.71878347e-05
 0.00000000e+00 6.75522626e-06 1.75611865e-05 9.07470025e-01
 1.44498862e-05 1.10635903e+00 0.00000000e+00 5.05883693e-06
 6.03671023e-01 4.46866466e-06 8.87735553e-01 9.98802459e-01
 0.00000000e+00 0.00000000e+00 4.03214599e-05 1.85253693e-06
 5.10222327e-06 1.51515475e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.43339705e-06 0.00000000e+00 1.38332912e-06 4.03629591e-06
 0.00000000e+00 3.50783548e-06 0.00000000e+00 0.00000000e+00
 1.26140294e-06 0.00000000e+00 1.67897105e-06 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.49824229, 0.        , 0.79320522, 0.91384239, 0.88796504],
       [1.24654892, 0.        , 0.        , 0.        , 0.        ],
       [0.90747003, 0.        , 1.10635903, 0.        , 0.        ],
       [0.60367102, 0.        , 0.88773555, 0.99880246, 0.        ]])

# New Stuff for NN

In [24]:
import tensorflow as tf
import random
# Parameters
learning_rate = 0.001
num_steps = 2500
batch_size = 32
display_step = 100

# Network Parameters

num_inputs = 5
n_hidden = 5
num_outputs = 1 #regresion output

# tf Graph input
X = tf.placeholder("float", [None, num_inputs])
Y = tf.placeholder("float", [None, num_outputs])


# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_inputs, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, num_outputs]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([num_outputs]))
}

def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

def regularization_loss(X):
    W = weights['h1'] # YAO: I am unclear if we want the edge weights h1 or out...
    '''
    M = X @ W
    if loss_type == 'l2':
        R = X - M
        loss = 0.5 / X.shape[0] * (R ** 2).sum()
        G_loss = - 1.0 / X.shape[0] * X.T @ R
    '''
    M = X @ W
    R = X - M
    mse_loss = tf.reduce_mean(tf.square(R))
    '''
    M = np.eye(d) + W * W / d  # (Yu et al. 2019)
    E = np.linalg.matrix_power(M, d - 1)
    h = (E.T * M).sum() - d
    '''
    n, d = X.shape
    M = tf.convert_to_tensor(np.eye(d))
    d = tf.cast(d, tf.float32)
    M = M + tf.cast(W * W / d, tf.float64)
    E = tf.pow(M, tf.cast(d - 1, tf.float64))

    h = tf.reduce_sum(tf.cast(tf.transpose(E) * M, tf.float32)) - d 

    '''Some NOTEARS params'''
    rho = 1 # I don't think we need to do anything with this.
    alpha = 1 #same here
    lambda1 = 0.1 

    '''
    obj = loss + 0.5 * rho * h * h + alpha * h + lambda1 * w.sum()
    '''
    custom_loss = mse_loss + 0.5 * h * h * rho + alpha * h + lambda1 * tf.reduce_sum(W) 
    return custom_loss


# Construct model
model = neural_net(X)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
loss_op = optimizer.minimize(regularization_loss(X))
init = tf.global_variables_initializer()

In [25]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x = np.array(random.choices(X_DAG, k=batch_size))
        batch_y = np.zeros((32, 1)) # For now this 'Y does nothing'
        sess.run(loss_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            _, loss = sess.run([loss_op, regularization_loss(X)], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(loss)) 

    print("Optimization Finished!")

    print(weights['h1'].eval())
    print(nx.adjacency_matrix(G).todense())

Step 1, Minibatch Loss= 211.3484
Step 100, Minibatch Loss= 168.0577
Step 200, Minibatch Loss= 260.8247
Step 300, Minibatch Loss= 121.1237
Step 400, Minibatch Loss= 85.0117
Step 500, Minibatch Loss= 66.0043
Step 600, Minibatch Loss= 55.5152
Step 700, Minibatch Loss= 67.8782
Step 800, Minibatch Loss= 40.3970
Step 900, Minibatch Loss= 22.0684
Step 1000, Minibatch Loss= 31.6012
Step 1100, Minibatch Loss= 22.7454
Step 1200, Minibatch Loss= 21.1569
Step 1300, Minibatch Loss= 14.8793
Step 1400, Minibatch Loss= 12.5130
Step 1500, Minibatch Loss= 11.9307
Step 1600, Minibatch Loss= 7.0512
Step 1700, Minibatch Loss= 6.7492
Step 1800, Minibatch Loss= 5.6420
Step 1900, Minibatch Loss= 5.7373
Step 2000, Minibatch Loss= 4.5596
Step 2100, Minibatch Loss= 4.3620
Step 2200, Minibatch Loss= 4.3702
Step 2300, Minibatch Loss= 3.3913
Step 2400, Minibatch Loss= 3.1503
Step 2500, Minibatch Loss= 2.8304
Optimization Finished!
[[ 0.4915184  -0.26115128 -0.25499156  0.60967195  0.9604991 ]
 [ 0.54276454  0.02792